In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#reading data
data = pd.read_csv(r'../Internship/Q2-tennis.csv')

In [3]:
def inputData(data):
    #60-40 split
    separatingLen = int(len(data)*.6)
    
    trainingData = data[:separatingLen]
    testData = data[separatingLen:]
    
    #extracting features and target
    features = list(data)
    target = features[4]
    outcomes = set(data[target])
    features.remove(target)
    
    #calculating prior probability 
    priorProb = {}
    for i in outcomes:
        priorProb[i] = 0
    for i in (trainingData[target]):
        for j in outcomes:
            if i == j:
                priorProb[i] +=1
    for i in outcomes:
        priorProb[i] /= len(trainingData)  
    
    #calculating conditional probability for features
    yes,no = [],[]
    for i in range(len(trainingData)):
        if trainingData[target][i]=='yes':
            yes.append(i)
        else :
            no.append(i) 
    return (features,trainingData,testData,priorProb,yes,no)           

In [4]:
def countInstances(features,trainingData,yes,no):
    outcomeVals1 ={}
    outcomeVals2 ={}
    for i in features:
        for j in list(set(data[i])):
            outcomeVals1[j] = 0
            outcomeVals2[j] = 0
    for i in yes:
        for j in features:
            if trainingData[j][i] in outcomeVals1:
                outcomeVals1[trainingData[j][i]]+=1
    for i in no:
        for j in features:
            if trainingData[j][i] in outcomeVals2:
                outcomeVals2[trainingData[j][i]]+=1
    return (outcomeVals1,outcomeVals2)        
            

In [5]:
def laplacianSmoothing(outcomeVals1,outcomeVals2,alpha = 1):
    #I have taken alpha as 1 by default
    count = 0
    for i in outcomeVals1.keys():
        if outcomeVals1[i]==0:
            count+=1
    for i in outcomeVals2.keys():
        if outcomeVals2[i]==0:
            count+=1
    #print(count)        
    if count>0:
        for i in outcomeVals1.keys():
            outcomeVals1[i]+=alpha
        for i in outcomeVals2.keys():
            outcomeVals2[i]+=alpha
    
    total1 = sum(outcomeVals1.values())
    total2 = sum(outcomeVals2.values())
    for i in outcomeVals1.keys():
        outcomeVals1[i] = outcomeVals1[i]/total1
    for i in outcomeVals2.keys():
        outcomeVals2[i] = outcomeVals2[i]/total2
     
    return (outcomeVals1,outcomeVals2)

In [6]:
def naiveBayes(testData,i,outcomeVals1,outcomeVals2,priorProb):
    val1 = priorProb['yes']*outcomeVals1[testData.iloc[i]['Outlook']]*outcomeVals1[testData.iloc[i]['Temp.']]*outcomeVals1[testData.iloc[i]['Humidity']]*outcomeVals1[testData.iloc[i]['Windy']]
    val2 = priorProb['no']*outcomeVals2[testData.iloc[i]['Outlook']]*outcomeVals2[testData.iloc[i]['Temp.']]*outcomeVals2[testData.iloc[i]['Humidity']]*outcomeVals2[testData.iloc[i]['Windy']]
    if (val1>val2):
        #print(val1,val2)
        return 'yes'
    else:
        return 'no'
   

In [7]:
def gettingPredictions(testData,outcomeVals1,outcomeVals2,priorProb):
    testData['Predictions']  = 'NA'
    num = len(testData)
    outcomeList = []
    for i in range(len(testData)):
        outcomeList.append(naiveBayes(testData,i,outcomeVals1,outcomeVals2,priorProb))
    testData.loc[:]['Predictions'] = outcomeList
    return testData

In [8]:
def main(alpha):
    data = pd.read_csv(r'../Internship/Q2-tennis.csv')
    
    #extraction
    features,trainingData,testData,priorProb,yes,no=inputData(data)
    
    #processing
    outcomeVals1,outcomeVals2=countInstances(features,trainingData,yes,no)
    outcomeVals1,outcomeVals2=laplacianSmoothing(outcomeVals1,outcomeVals2,alpha)
    
    #getting output
    return gettingPredictions(testData,outcomeVals1,outcomeVals2,priorProb)
    
     

In [11]:
# select an integer value for alpha of Laplacian Smoothing
alpha = int(input('Enter an int value for alpha\n'))

Enter an int value for alpha
1


In [12]:
main(alpha)

,Outlook,Temp.,Humidity,Windy,Play,Predictions
8,sunny,cool,normal,false,yes,no
9,rainy,mild,normal,false,yes,yes
10,sunny,mild,normal,true,yes,no
11,overcast,mild,high,true,yes,yes
12,overcast,hot,normal,false,yes,yes
13,rainy,mild,high,true,no,no
